In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import Activation, BatchNormalization, Add
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
import os
import pandas as pd

drive_dir = '/content/drive/My Drive/'
data_dir = drive_dir + 'OCT/'
rnfl_dir = data_dir + 'RNFLT/'
scan_dir = data_dir + 'Bscan/'
pre_dir = data_dir + 'proc/'

data = pd.read_csv(data_dir + 'ReadMe/data_summary.csv')

imgs, labels, count = [], [], 0
for fn in sorted(os.listdir(pre_dir)):
  if count < 600:
    print(fn)
    img = np.load(pre_dir + fn)
    img = img['bscans']

    for i in range(200):
      labels.append(data['glaucoma'][count])

    for im in img:
      imgs.append(im)

  count += 1

print(len(imgs), len(labels))
imgs = np.array(imgs)
labels = np.array(labels)

data_0001.npz
data_0002.npz
data_0003.npz
data_0004.npz
data_0005.npz
data_0006.npz
data_0007.npz
data_0008.npz
data_0009.npz
data_0010.npz
data_0011.npz
data_0012.npz
data_0013.npz
data_0014.npz
data_0015.npz
data_0016.npz
data_0017.npz
data_0018.npz
data_0019.npz
data_0020.npz
data_0021.npz
data_0022.npz
data_0023.npz
data_0024.npz
data_0025.npz
data_0026.npz
data_0027.npz
data_0028.npz
data_0029.npz
data_0030.npz
data_0031.npz
data_0032.npz
data_0033.npz
data_0034.npz
data_0035.npz
data_0036.npz
data_0037.npz
data_0038.npz
data_0039.npz
data_0040.npz
data_0041.npz
data_0042.npz
data_0043.npz
data_0044.npz
data_0045.npz
data_0046.npz
data_0047.npz
data_0048.npz
data_0049.npz
data_0050.npz
data_0051.npz
data_0052.npz
data_0053.npz
data_0054.npz
data_0055.npz
data_0056.npz
data_0057.npz
data_0058.npz
data_0059.npz
data_0060.npz
data_0061.npz
data_0062.npz
data_0063.npz
data_0064.npz
data_0065.npz
data_0066.npz
data_0067.npz
data_0068.npz
data_0069.npz
data_0070.npz
data_0071.npz
data_0

In [ ]:
from keras.applications import VGG16

image_size = (200, 300)
batch_size = 32
epochs = 10
num_classes = 1

vgg = VGG16(weights='imagenet', include_top=False, input_shape=(200, 300, 3))

#vgg.trainable = True

single = Input(shape=(200, 300, 1))
x = Conv2D(3, (1, 1), padding='same')(single)
vgg_output = vgg(x)

gap = GlobalAveragePooling2D()(output)
final = Dense(1, activation='sigmoid')(gap)

model = Model(inputs=single, outputs=final)
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy',tf.keras.metrics.AUC(),
                tf.keras.metrics.Recall(),
                tf.keras.metrics.Precision()])
model.summary()

58889256/58889256 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 200, 300, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 200, 300, 3)       6         
                                                                 
 vgg16 (Functional)          (None, 6, 9, 512)         14714688  
                                                                 
 global_average_pooling2d (  (None, 512)               0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 1)                 513       
                                                                 
Total params: 14715207 (56.13 MB)
Trainable params: 14715207 (

In [ ]:
from sklearn.model_selection import train_test_split

filepath = "/content/drive/MyDrive/OCT/models/vgg/glauc-epoch-{epoch:02d}.hdf5"

cb = [
    ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=False, mode='max', save_freq='epoch')
]

X_train, X_val, y_train, y_val = train_test_split(imgs, labels, test_size=0.2, random_state=42)

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)

(96000, 200, 300) (96000,)
(24000, 200, 300) (24000,)


In [ ]:
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_val, y_val), callbacks=[cb])

Epoch 1/10
 367/3000 [==>...........................] - ETA: 11:50 - loss: 2.0739 - accuracy: 0.5547 - auc: 0.5088 - recall: 0.0866 - precision: 0.4772

In [ ]:
model.history.history
keys = list(model.history.history.keys())
keys

In [ ]:
train_loss = model.history.history[keys[0]]
print(keys[0])

val_loss = model.history.history[keys[5]]
print(keys[5])

train_accuracy = model.history.history[keys[1]]
print(keys[1])

val_accuracy = model.history.history[keys[6]]
print(keys[6])

train_auc = model.history.history[keys[2]]
print(keys[2])

val_auc = model.history.history[keys[7]]
print(keys[7])

train_precision = model.history.history[keys[3]]
print(keys[3])

val_precision = model.history.history[keys[8]]
print(keys[8])

train_recall = model.history.history[keys[4]]
print(keys[4])

val_recall = model.history.history[keys[9]]
print(keys[9])

epochs = range(1, len(train_loss) + 1)

In [ ]:
import matplotlib.pyplot as plt

results_dir = '/content/drive/MyDrive/OCT/models/vgg/results/'

plt.plot(epochs, train_loss, label='Train')
plt.plot(epochs, val_loss, label='Validation')
plt.title('Loss over time')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig(results_dir + 'loss.png')
plt.show()

plt.plot(epochs, train_accuracy, label='Train')
plt.plot(epochs, val_accuracy, label='Validation')
plt.title('Accuracy over time')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig(results_dir + 'accuracy.png')
plt.ylim(0, 1)
plt.show()

plt.plot(epochs, train_auc, label='Traini')
plt.plot(epochs, val_auc, label='Validation')
plt.title('AUC over time')
plt.xlabel('Epochs')
plt.ylabel('AUC')
plt.legend()
plt.savefig(results_dir + 'auc.png')
plt.ylim(0, 1)
plt.show()

plt.plot(epochs, train_precision, label='Train')
plt.plot(epochs, val_precision, label='Validation')
plt.title('Precision over time')
plt.xlabel('Epochs')
plt.ylabel('Precision')
plt.legend()
plt.savefig(results_dir + 'precision.png')
plt.ylim(0, 1)
plt.show()

plt.plot(epochs, train_recall, label='Train')
plt.plot(epochs, val_recall, label='Validation')
plt.title('Recall over time')
plt.xlabel('Epochs')
plt.ylabel('Recall')
plt.legend()
plt.savefig(results_dir + 'recall.png')
plt.ylim(0, 1)
plt.show()